In [1]:
#Exercise 1: Calculating Required Sample Size

from statsmodels.stats.power import TTestIndPower

# define the parameters
effect_size = 0.3
power = 0.8
alpha = 0.05

#calculate the required sample size
analysis = TTestIndPower()
sample_size = analysis.solve_power(effect_size=effect_size, alpha=alpha, power=power)

print(f'The required sample size per group: {sample_size:.2f}') #175.38 / group

The required sample size per group: 175.38


In [3]:
#Exercise 2: Understanding the Relationship Between Effect Size and Sample Size


# 0.2 effect size
effect_size_one = 0.2
effect_size_two = 0.4
effect_size_three = 0.5

sample_size_one = analysis.solve_power(effect_size=effect_size_one, alpha=alpha, power=power)
sample_size_two = analysis.solve_power(effect_size=effect_size_two, alpha=alpha, power=power)
sample_size_three = analysis.solve_power(effect_size=effect_size_three, alpha=alpha, power=power)

print(f'The required sample size per group: \nSample Size 0.2: {sample_size_one:.2f},\nSample Size 0.4:{sample_size_two:.2f},\nSample Size 0.5: {sample_size_three:.2f}')

#Analysis: As effect size increases, the sample size decreases from 393.4 to 99.08 to 63. This is because larger effect sizes are easier to detect.

The required sample size per group: 
Sample Size 0.2: 393.41,
Sample Size 0.4:99.08,
Sample Size 0.5: 63.77


In [4]:
#🌟 Exercise 3: Exploring the Impact of Statistical Power

#effect_size = 0.2
alpha = 0.05

powers = [0.7, 0.8, 0.9]

for p in powers:
  n = analysis.solve_power(effect_size=effect_size, alpha=alpha,power=p)
  print(f"Power{p}: required sample size / group is {n:.2f}.")

  ##Analaysis: As power incr. sample size also increases which means we need more data to be sure. It means we need to keep thi sin mind when designing an A/B test to properly plan our controls for the test aganst our resourcing capabilities.

Power0.7: required sample size / group is 138.12.
Power0.8: required sample size / group is 175.38.
Power0.9: required sample size / group is 234.46.


In [5]:
#Exercise 4: Implementing Sequential Testing
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

alpha = 0.05
#stoping criteria is when p-value is < 0.05 and max number of weeks is 4
weekly_data = {
    1: {"A": (30,1000), "B": (42, 1000)},
    2: {"A": (55, 2000), "B": (75, 2000)},
    3: {"A": (79, 3000), "B": (103, 3000)},
}

for week in weekly_data:
    conv_A, total_A = weekly_data[week]["A"]
    conv_B, total_B = weekly_data[week]["B"]

    count = np.array([conv_A, conv_B])
    nobs = np.array([total_A, total_B])

    stat, p_value = proportions_ztest(count, nobs)

    print(f"Week {week}: p-value = {p_value:.4f}")

    # Check stopping rule
    if p_value < alpha and (conv_B / total_B) > (conv_A / total_A):
        print(f"Stop at week {week}: Version B is significantly better.\n")
        break
else:
    print("No early stop — continue test or evaluate at final week.")


    #Analaysis: if at 0.02, i'd continue, because it means that it still has more to go below my criteria.

Week 1: p-value = 0.1498
Week 2: p-value = 0.0745
Week 3: p-value = 0.0708
No early stop — continue test or evaluate at final week.


In [ ]:
#Exercise 5: Applying Bayesian A/B Testing

##You’re testing a new feature in your app, and you want to use a Bayesian approach. Initially, you believe the new feature has a 50% chance of improving user engagement. After collecting data, your analysis suggests a 65% probability that the new feature is better.

    ###Describe how you would set up your prior belief: I believe the 50% chance is net neutral so i don't need to set up anything new.
    ##After collecting data, how does the updated belief (posterior distribution) influence your decision?: THere is some evidence its better but honestly not strong enough to show that the new feature is SO much better. We need to continue the test.
    ###What would you do if the posterior probability was only 55%?: This is not strong enough to say that th enew feature is better and i would not use this as evidence at all. i may look at nother varabiel instead.


In [ ]:
#Exercise 6: Implementing Adaptive Experimentation

##You’re running a test with three different website layouts to increase user engagement. Initially, each layout gets 33% of the traffic. After the first week, Layout C shows higher engagement.

###Explain how you would adjust the traffic allocation after the first week: I wouldn't do anything just yet until I see much higher engagement (like 75%+ ). If its consistent, then I would begin shifting by 10% more each week.

###Describe how you would continue to adapt the experiment in the following weeks: Divert from 33 - 43% of teh traffic in week 3 (if the first two weeks are execessively high). Reduce the traffic by 5% to a an db.

###What challenges might you face with adaptive experimentation, and how would you address them? : It could be confirmation bias, so we need to shift slowly.
